In [1]:
# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import statsmodels.formula.api as smf 
from scipy.interpolate import interp1d
import seaborn.objects as so
from sklearn.preprocessing import StandardScaler

In [2]:
cancellations_england_fp = os.path.join(os.getcwd(), "datasets", "nhsenglandcancellation.xlsx")
cancellations_england = pd.read_excel(cancellations_england_fp)

In [3]:
cancellations_england.drop(columns=["Unnamed: 0","Patients not treated within 28 days of cancellation.","Patients not treated within 28 days of cancellation as percentage of cancellations"], inplace=True)

In [4]:
cancellations_england

,Year,Quarter,Number of cancelled elective operations,Elective \nadmissions,Cancelled operations as percentage of elective admissions
0,1994/95,1,11603.0,1.054818e+06,0.011000
1,1994/95,2,10186.0,1.131778e+06,0.009000
2,1994/95,3,11874.0,1.187400e+06,0.010000
3,1994/95,4,14466.0,1.205500e+06,0.012000
4,1995/96,1,11860.0,1.186000e+06,0.010000
...,...,...,...,...,...
114,2022/23,3,21273.0,1.904705e+06,0.011168
115,2022/23,4,18975.0,1.987581e+06,0.009546
116,2023/24,1,17024.0,1.948942e+06,0.008734
117,2023/24,2,18749.0,1.990449e+06,0.009419


In [5]:
cancellations_england.isna().count()
# so all every cell have a data

Year                                                         119
Quarter                                                      119
Number of cancelled elective operations                      119
Elective \nadmissions                                        119
Cancelled operations as percentage of elective admissions    119
dtype: int64

In [6]:
# delete rows before the year 2014/2015
cancellations_england = cancellations_england[cancellations_england["Year"] >= "2013/2014"]

In [7]:
cancellations_england

,Year,Quarter,Number of cancelled elective operations,Elective \nadmissions,Cancelled operations as percentage of elective admissions
80,2014/15,1,15650.0,1966435.0,0.007959
81,2014/15,2,15898.0,1894377.0,0.008392
82,2014/15,3,19470.0,1904502.0,0.010223
83,2014/15,4,20464.0,1927719.0,0.010616
84,2015/16,1,16099.0,1915905.0,0.008403
85,2015/16,2,16414.0,1968406.0,0.008339
86,2015/16,3,18393.0,1966783.0,0.009352
87,2015/16,4,23352.0,1925765.0,0.012126
88,2016/17,1,18730.0,1980104.0,0.009459
89,2016/17,2,19446.0,1994782.0,0.009748


In [8]:
cancellations_scotland_fp = os.path.join(os.getcwd(), "datasets", "cancellations_in_scotland.csv")
cancellations_scotland = pd.read_csv(cancellations_scotland_fp)

In [10]:
cancellations_scotland

,_id,Month,Country,TotalOperations,TotalOperationsQF,TotalCancelled,TotalCancelledQF,CancelledByPatientReason,CancelledByPatientReasonQF,ClinicalReason,ClinicalReasonQF,NonClinicalCapacityReason,NonClinicalCapacityReasonQF,OtherReason,OtherReasonQF
0,1,201505,S92000003,28810,NaN,2695,NaN,1057,NaN,853,NaN,523,NaN,262,NaN
1,2,201506,S92000003,30941,NaN,2800,NaN,1194,NaN,870,NaN,449,NaN,287,NaN
2,3,201507,S92000003,27725,NaN,2642,NaN,1083,NaN,854,NaN,457,NaN,248,NaN
3,4,201508,S92000003,28948,NaN,2630,NaN,1080,NaN,912,NaN,491,NaN,147,NaN
4,5,201509,S92000003,30938,NaN,2829,NaN,1122,NaN,1000,NaN,535,NaN,172,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,101,202309,S92000003,23209,NaN,2105,NaN,739,NaN,738,NaN,541,NaN,87,NaN
101,102,202310,S92000003,24201,NaN,2286,NaN,775,NaN,797,NaN,580,NaN,134,NaN
102,103,202311,S92000003,25922,NaN,2277,NaN,737,NaN,820,NaN,582,NaN,138,NaN
103,104,202312,S92000003,20837,NaN,2014,NaN,677,NaN,725,NaN,515,NaN,97,NaN


In [11]:
cancellations_scotland["Month"] = pd.to_datetime(cancellations_scotland.Month, format="%Y%m")


In [13]:
cancellations_scotland.drop(columns=["Country","_id","TotalOperationsQF","TotalCancelledQF","CancelledByPatientReasonQF","ClinicalReasonQF","NonClinicalCapacityReasonQF","OtherReasonQF"], inplace=True)

In [15]:
cancellations_scotland.rename(columns={"Month":"Date"}, inplace=True)

In [16]:
cancellations_scotland

,Date,TotalOperations,TotalCancelled,CancelledByPatientReason,ClinicalReason,NonClinicalCapacityReason,OtherReason
0,2015-05-01,28810,2695,1057,853,523,262
1,2015-06-01,30941,2800,1194,870,449,287
2,2015-07-01,27725,2642,1083,854,457,248
3,2015-08-01,28948,2630,1080,912,491,147
4,2015-09-01,30938,2829,1122,1000,535,172
...,...,...,...,...,...,...,...
100,2023-09-01,23209,2105,739,738,541,87
101,2023-10-01,24201,2286,775,797,580,134
102,2023-11-01,25922,2277,737,820,582,138
103,2023-12-01,20837,2014,677,725,515,97
